In [1]:
import numpy as np
import plotly.express as px
import math
import plotly.figure_factory as ff
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from tabulate import tabulate

In [145]:
def simulador(lambda_=1, mi_=2, simulation_total_time=1000, deterministic=False, initial_customers=1, custom_limited = False):
    n = initial_customers
    custom_limited = 1 if custom_limited else 0
    simul_time = number_of_arrivals = system_spent_time = clients_in_system = clients_in_line = 0
    line_queue, wait_times, occupied_periods = [], [], []
    time_departures, time_arrivals = [], []
    occupied_period_start = None
    occupied = True
    
    exponential = (lambda mi: 1 / mi) if deterministic else (lambda mi: np.random.exponential(scale=1 / mi))

    while simul_time < simulation_total_time:
        arrival_time = np.random.exponential(scale=1 / lambda_)  # Tempo de chegada exponencialmente distribuído
        departure_time = exponential(mi_)  # Tempo de saída exponencialmente distribuído
        last_event_time = simul_time  # Tempo do último evento (chegada ou saída)

        if n == initial_customers and occupied:
                occupied_period_start = simul_time  # Início do período ocupado
                occupied = False
                
        if n == custom_limited and not occupied:
                occupied_periods.append(simul_time - occupied_period_start)  # Fim do período ocupado
                occupied = True
                
        if n == 0 or arrival_time < departure_time:
            simul_time += arrival_time  # Incrementa o tempo total de simulação
            time_arrivals.append(simul_time)  # Registrar o tempo de chegada
            number_of_arrivals += 1  # Incrementar o número de chegadas
            clients_in_system += (simul_time - last_event_time) * n  # Atualizar o tempo total no sistema
            clients_in_line += (simul_time - last_event_time) * len(line_queue)  # Atualizar o tempo total na fila
            n += 1  # Incrementar o número de clientes no sistema

            if n > 1:
                line_queue.append(simul_time)  # Adicionar o tempo de chegada à fila

        else:
            simul_time += departure_time  # Incrementa o tempo total de simulação
            time_departures.append(simul_time)  # Registrar o tempo de saída
            clients_in_system += (simul_time - last_event_time) * n  # Atualizar o tempo total no sistema
            clients_in_line += (simul_time - last_event_time) * len(line_queue)  # Atualizar o tempo total na fila

            if len(line_queue) > 0:
                wait_times.append(simul_time - line_queue.pop(0))  # Calcular o tempo de espera do cliente que saiu da fila
            n -= 1  # Decrementar o número de clientes no sistema
            if len(time_arrivals) > 0:
                system_spent_time += simul_time - time_arrivals.pop(0)  # Calcular o tempo total gasto no sistema

            

    avg_waiting_time = sum(wait_times) / number_of_arrivals  # Calcular o tempo médio de espera
    avg_system_time = system_spent_time / number_of_arrivals  # Calcular o tempo médio no sistema
    avg_clients_in_system = clients_in_system / simul_time  # Calcular a média de clientes no sistema
    avg_clients_in_line = clients_in_line / simul_time  # Calcular a média de clientes na fila
    avg_number_of_departure = len(time_departures) / simul_time  # Calcular a média de partidas por unidade de tempo
    avg_occupied_period_duration = sum(occupied_periods) / len(occupied_periods) if len(occupied_periods) > 0 else 0  # Calcular a duração média do período ocupado

    results = {
        'people_in_line': n,
        'simul_time': simul_time,
        'avg_waiting_time': avg_waiting_time,
        'avg_system_time': avg_system_time,
        'number_of_arrivals': number_of_arrivals,
        'avg_clients_in_system': avg_clients_in_system,
        'avg_clients_in_line': avg_clients_in_line,
        'wait_times': wait_times,
        'number_of_departure': avg_number_of_departure,
        'time_between_departures': time_departures,
        'avg_occupied_period_duration': avg_occupied_period_duration
    }

    return results


In [159]:
# obtendo dados simulados
data = simulador(lambda_= 1, mi_= 2, simulation_total_time = 20000, initial_customers=0, custom_limited=False)

table_data = [
    ["Total de pessoas que foram ao banco na simulação", f"{data['number_of_arrivals']}"],
    ["Número de pessoas na fila ao final da simulação", f"{data['people_in_line']}"],
    ["Tempo total da simulação", f"{data['simul_time']:.4f}"],
    ["Tempo médio do cliente no sistema", f"{data['avg_system_time']:.4f}"],
    ["Tempo médio de espera na fila", f"{data['avg_waiting_time']:.4f}"],
    ["Número médio de clientes no sistema", f"{data['avg_clients_in_system']:.4f}"],
    ["Número médio de clientes na fila", f"{data['avg_clients_in_line']:.4f}"],
    ["Duração média do período ocupado", f"{data['avg_occupied_period_duration']:.4f}"]
]

headers = ["Descrição", "Valor"]
print(tabulate(table_data, headers=headers, tablefmt="grid",stralign="left", numalign="right"))

+--------------------------------------------------+---------+
| Descrição                                        |   Valor |
+==================================================+=========+
| Total de pessoas que foram ao banco na simulação |   19961 |
+--------------------------------------------------+---------+
| Número de pessoas na fila ao final da simulação  |       2 |
+--------------------------------------------------+---------+
| Tempo total da simulação                         | 20000.6 |
+--------------------------------------------------+---------+
| Tempo médio do cliente no sistema                |   0.993 |
+--------------------------------------------------+---------+
| Tempo médio de espera na fila                    |  0.4956 |
+--------------------------------------------------+---------+
| Número médio de clientes no sistema              |  0.9911 |
+--------------------------------------------------+---------+
| Número médio de clientes na fila                 |  0

In [27]:
C_values = [ i for i in range(1, 11)]  # Valores de C de 1 a 10
simulator_results = []                 # Média dos resultados simulados para cada C
expected_results = []                  # Resultados Analiticos para cada C
std_simulator = []                     # Desvio padrão das execuções para cada C
simulator_distribution = {}                      # Resultados das simulações para cada C

for C in C_values:
    result = [simulador(lambda_=1, mi_=2, simulation_total_time=10000, deterministic=False, initial_customers=C)['avg_occupied_period_duration'] for _ in range(100)]
    std_simulator.append(np.std(result))
    simulator_results.append(np.mean(result))
    expected_results.append(C * 0.5 / (1 - 0.5))  # Calcula o valor esperado usando a fórmula
    simulator_distribution[C] = result

In [28]:
distribution_data = simulator_distribution[2]

fig = ff.create_distplot([distribution_data], [''], curve_type='normal', bin_size=0.04)

fig.update_layout(
    xaxis=dict(title='Tempo', showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(title='Quantidade', zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)
fig.show()

In [29]:
fig = go.Figure(go.Histogram(x=distribution_data, nbinsx=20))

fig.update_layout(
    xaxis=dict(title='Tempo'),
    yaxis=dict(title='Quantidade')
)
fig.update_layout(
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)
fig.show()

In [30]:
fig = go.Figure(data=[go.Box(y=simulator_distribution[C], name=f'C={C}') for C in C_values])
fig.update_layout(
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)
fig.show()

In [31]:
def get_z_score(confidence_level):
    confidence_levels = {0.9: 1.645, 0.95: 1.96, 0.99: 2.58}
    return confidence_levels.get(confidence_level, None)
    
def calculate_confidence_interval(errors, sample_size, confidence_level):
    z = get_z_score(confidence_level)
    if z is None:
        return None

    return [z * (s / (sample_size ** 0.5)) for s in errors]

In [126]:
n_sample = len(next(iter(simulator_distribution.values())))
control_interval = calculate_confidence_interval(std_simulator, n_sample, 0.95)

fig = go.Figure(data=[
    go.Bar(
        name='Valor Simulador',
        x=C_values,
        y=simulator_results,
        error_y=dict(type='data', array=control_interval),
    ),
    go.Bar(
        name='Valor Analítico',
        x=C_values,
        y=expected_results,
    )
])

fig.update_layout(
    title='Duração média do período ocupado generalizado',
    xaxis=dict(title='C', showgrid=False, zeroline=False, showticklabels=True),
    yaxis=dict(title='E(B_C)', zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)
fig.show()

In [33]:
fig = go.Figure()

# Adicionar os resultados simulados
fig.add_trace(
    go.Scatter(
        x=C_values,
        y=simulator_results,
        error_y=dict(
            type='data',
            symmetric=True,
            array=control_interval,
            thickness=1,
            width=2
        ),
        mode='markers',
        name='Simulador'
    )
)

# Adicionar os valores esperados
fig.add_trace(
    go.Scatter(
        x=C_values,
        y=expected_results,
        mode='markers',
        name='Valor Esperado'
    )
)

# Configurar o layout
fig.update_layout(
    title='Duração média do período ocupado generalizado',
    xaxis=dict(title='C', showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(title='E(B_C)', zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)

fig.show()

In [34]:
fig = go.Figure()

for index, C in enumerate(C_values, start=1):
    data = simulator_distribution[C]
    bin = 14 if C < 7 else 20
    fig.add_trace(go.Histogram(x=data, nbinsx=bin, name=f'C = {C}'))

# Configurar o layout
fig.update_layout(
    barmode='overlay',
    title='Distribuição de C'
)

# Definir a opacidade para visualizar ambos os histogramas
fig.update_traces(opacity=0.75)
fig.update_layout(
    title_text="Distribuição de C",
    height=600,
    xaxis=dict(title='Tempo', showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(title='Quantidade', zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)
fig.show()


In [36]:
control_interval = calculate_confidence_interval(std_simulator, n_sample, 0.95)

fig = make_subplots(rows=4, cols=3, subplot_titles=[f'C = {C}' for C in C_values])

# Cálculo do intervalo de confiança e plotagem dos histogramas
for index, C in enumerate(C_values, start=1):
    row = (index - 1) // 3 + 1
    col = (index - 1) % 3 + 1
    
    # Cálculo do intervalo de confiança para a chave C
    data = simulator_distribution[C]
    x_barra = np.mean(data)

    limite_inferior = x_barra - control_interval[index-1]
    limite_superior = x_barra + control_interval[index-1]
    
    # Plotagem do histograma
    fig.add_trace(
        go.Histogram(x=data, nbinsx=14, name=f'C = {C}'),
        row=row, col=col
    )
    
    # Adição das linhas do intervalo de confiança
    fig.add_shape(
        type="line", x0=limite_inferior, y0=0, x1=limite_inferior, y1=30, row=row, col=col,
        line=dict(color='black', width=2, dash='dash')
    )
    fig.add_shape(
        type="line", x0=limite_superior, y0=0, x1=limite_superior, y1=30, row=row, col=col,
        line=dict(color='black', width=2, dash='dash')
    )

# fig.update_layout(height=600, title_text="Distribuição de C")

fig.update_layout(
    title_text="Distribuição de C",
    height=600,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)
fig.show()


In [37]:
def remove_outliers(data):
    q1 = np.quantile(data, 0.25)
    q3 = np.quantile(data, 0.75)
    iqr = q3 - q1

    upper_bound = q3 + (1.5 * iqr)
    lower_bound = q1 - (1.5 * iqr)

    data_without_outliers = [x for x in data if (x >= lower_bound) and (x <= upper_bound)]

    outlier_percentage = (len(data) - len(data_without_outliers)) / len(data) * 100

    return data_without_outliers, outlier_percentage

In [38]:
for C in C_values:
    _, percetage = remove_outliers(simulator_distribution[C])
    print(f"Quant. outliers removidos para (C = {C}): {percetage} %")

Quant. outliers removidos para (C = 1): 0.0 %
Quant. outliers removidos para (C = 2): 0.0 %
Quant. outliers removidos para (C = 3): 2.0 %
Quant. outliers removidos para (C = 4): 0.0 %
Quant. outliers removidos para (C = 5): 0.0 %
Quant. outliers removidos para (C = 6): 0.0 %
Quant. outliers removidos para (C = 7): 4.0 %
Quant. outliers removidos para (C = 8): 1.0 %
Quant. outliers removidos para (C = 9): 1.0 %
Quant. outliers removidos para (C = 10): 5.0 %


# 2.2

In [119]:
C_values = [ i for i in range(1, 11)]  # Valores de C de 2 a 10
simulator_results = []                 # Média dos resultados simulados para cada C
expected_results = []                  # Resultados Analiticos para cada C
std_simulator = []                     # Desvio padrão das execuções para cada C
simulator_distribution = {}                      # Resultados das simulações para cada C

for C in C_values:
    result = [simulador(lambda_=1, mi_=2, simulation_total_time=10000, deterministic=False, initial_customers=C, custom_limited=True)['avg_occupied_period_duration'] for _ in range(100)]
    std_simulator.append(np.std(result))
    simulator_results.append(np.mean(result))
    expected_results.append(C-1)  # Calcula o valor esperado usando a fórmula
    simulator_distribution[C] = result

In [125]:
n_sample = len(next(iter(simulator_distribution.values())))
control_interval = calculate_confidence_interval(std_simulator, n_sample, 0.95)

fig = go.Figure(data=[
    go.Bar(
        name='Valor Simulador',
        x=C_values,
        y=simulator_results,
        error_y=dict(type='data', array=control_interval),
    ),
    go.Bar(
        name='Valor Analítico',
        x=C_values,
        y=expected_results,
    )
])

fig.update_layout(
    title="Gráfico da Duração Média do Período Ocupado Generalizado para Finalização em 1",
    xaxis=dict(title='C', showgrid=False, zeroline=False, showticklabels=True),
    yaxis=dict(title='E(B_U)', zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)
fig.show()

In [144]:
fig = go.Figure()

# Adicionar os resultados simulados
fig.add_trace(
    go.Scatter(
        x=C_values,
        y=simulator_results,
        error_y=dict(
            type='data',
            symmetric=True,
            array=control_interval,
            thickness=1,
            width=2
        ),
        mode='markers',
        name='Simulador'
    )
)

# Adicionar os valores esperados
fig.add_trace(
    go.Scatter(
        x=C_values,
        y=expected_results,
        mode='markers',
        name='Valor Esperado'
    )
)

# Configurar o layout
fig.update_layout(
    title='Duração média do período ocupado generalizado',
    xaxis=dict(title='C', showgrid=False, zeroline=False, showticklabels=True),
    yaxis=dict(title='E(B_C)', zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)

fig.show()